# CNN model 
This code below is all about CNN model to classify MNIST datasets. It is demonstrating a shallow CNN model code simulation.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init as init

In [2]:
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
print('device: {}'.format(device))

device: cpu


In [3]:
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [4]:
learning_rate = 1e-3
training_epochs = 15
batch_size = 100

In [5]:
mnist_train = dsets.MNIST(root='MNIST_data/', train=True,  transform = transforms.ToTensor(),download=True)
mnist_test  = dsets.MNIST(root='MNIST_data/', train=False, transform = transforms.ToTensor(),download=True)

In [6]:
dataloader = DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle = True, drop_last = True)

In [17]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride = 2)
        ).to(device)
        self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
        ).to(device)
        self.fc = torch.nn.Linear(7*7*64, 10, bias=True).to(device)
        init.xavier_uniform_(self.fc.weight)
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        print(out.size())
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [8]:
model = CNN()

In [9]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(),lr = learning_rate)

In [10]:
iteration = len(dataloader)
print('iteration: {}'.format(iteration))

iteration: 600


In [16]:
for epoch in range(training_epochs):
    avg_cost = 0
    
    for index, (X, Y) in enumerate(dataloader):
        X = X.to(device)
        Y = Y.to(device)
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / iteration
        
    print('[Epoch: {:4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

TypeError: 'builtin_function_or_method' object is not iterable

In [ ]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: {}'.format(accuracy.item()))